In [60]:
import pandas as pd
#pip install openpyxl
#https://stackoverflow.com/questions/75440354/why-does-pandas-read-excel-fail-on-an-openpyxl-error-saying-readonlyworksheet
#pip install --force-reinstall -v "openpyxl==3.1.0"
#restart kernel

In [61]:
df = pd.read_excel('../Data/artists_list.xlsx')

column_names = df.columns.tolist()
print(column_names)

['nice to see you here 🦜🦾', 'This data is maintained by @sureailabs, @proximasan, @EErratica, and @KyrickYoung as part our ongoing Image Synthesis Style Studies project, started in Feb 2022. Its purpose is to document artist names and other prompt modifier keywords (see tabs) that are recognized by AI text-to-image synthesis models, such as the CLIP models used in Disco Diffusion and Stable Diffusion. We are not afilliated with the development or training of these models.\n\nYou can view outputs and more information about this project is available at parrotzone.art 🦜🦾', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', "Do NOT request edit permissions. We do not grant these. If you'd like to recommend a modifier not already on this sheet (to verify, search with CTRL+F), you can use this form: 🎟️ Artist Recommendations \n\nAll styles tested for CLIP in Disco Diffusion, v4.1, but should have similarly successful results in any platform where OpenAI's CLI

In [62]:
df = pd.read_excel('../Data/artists_list.xlsx', usecols=['Unnamed: 2', 'Unnamed: 7'])
print(df.head())

           Unnamed: 2                                  Unnamed: 7
0                 NaN  Other modifier studies: 767\n(CLIP, tab 2)
1     Copy/Paste Name                   Tags \n(work-in-progress)
2         Alvar Aalto                 architecture, high contrast
3         Slim Aarons                 vibrant, photography, scene
4  Edwin Austin Abbey                              scene, realism


In [63]:
#rename the column names
df.columns = ['artists', 'tags']

#and statrt from row 2
df = df.iloc[2:]

print(df.head())

                artists                         tags
2           Alvar Aalto  architecture, high contrast
3           Slim Aarons  vibrant, photography, scene
4    Edwin Austin Abbey               scene, realism
5  Gertrude Abercrombie            surrealism, dark,
6      Marina Abramović           scene, photography


In [64]:
#new columns and create new excel
df.to_excel('../Data/artists_list_new.xlsx', index=False)

In [65]:
############################
#read new excel
dfNew = pd.read_excel('../Data/artists_list_new.xlsx')
print(dfNew.head())
#first should be Alvar Aalto; architecture, high contrast

                artists                         tags
0           Alvar Aalto  architecture, high contrast
1           Slim Aarons  vibrant, photography, scene
2    Edwin Austin Abbey               scene, realism
3  Gertrude Abercrombie            surrealism, dark,
4      Marina Abramović           scene, photography


In [66]:
############################
#read styles excel
dfStyles = pd.read_excel('../Data/styles_list.xlsx')
print(dfStyles.head())

             Afrofuturism
0               ASCII art
1            Abstract art
2                Art Brut
3  Abstract expressionism
4    Abstract illusionism


In [67]:
#rename column
dfStyles.columns = ['styles']
print(dfStyles.head())

#lowercase
dfStyles = dfStyles.applymap(lambda x: x.lower())
print(dfStyles.head())

                   styles
0               ASCII art
1            Abstract art
2                Art Brut
3  Abstract expressionism
4    Abstract illusionism
                   styles
0               ascii art
1            abstract art
2                art brut
3  abstract expressionism
4    abstract illusionism


In [68]:
##################################
#convert floats and NaN to string
print(dfNew['tags'].dtype)
mask = dfNew['tags'].isna()
print(dfNew[mask])
dfNew.loc[mask, 'tags'] = dfNew.loc[mask, 'tags'].astype(str)

object
               artists tags
353      Jonas Burgert  NaN
471       Jean Cocteau  NaN
472    Clifford Coffin  NaN
473    Gregory Colbert  NaN
499       Dave Coverly  NaN
...                ...  ...
3480       Yosuke Ueno  NaN
3481  George Underwood  NaN
3482     Kelly Vivanco  NaN
3484    Angelina Wrona  NaN
3485  Vania Zouravliov  NaN

[976 rows x 2 columns]


In [69]:
######################################
#pip install pandasql
#pip install --upgrade SQLAlchemy==1.4.46
#https://stackoverflow.com/questions/75315117/attributeerror-connection-object-has-no-attribute-connect-when-use-df-to-sq
#since 2.0 throws errors
from pandasql import sqldf

#split tags in rows
dfNewSplit = dfNew.assign(tags=dfNew.tags.str.split(", ")).explode('tags').reset_index(drop=True)

#query as a string
query = """
SELECT artists, GROUP_CONCAT(tags, ', ') as tags
FROM (SELECT artists, tags
    FROM dfNewSplit a
    JOIN dfStyles s
    ON s.styles LIKE '%' || a.tags || '%'
    and a.tags != '-' and a.tags != ''
    GROUP BY 1, 2
)
GROUP BY artists
"""

#execute query and store the result in a new dataframe
filtered_dfNew = sqldf(query)

#write the result to an Excel file
filtered_dfNew.to_excel('../Data/filtered_artists_list_new.xlsx', index=False)

#print the result
print(filtered_dfNew.head())


                   artists                    tags
0            Aaron Douglas                  cubism
1           Aaron Jasinski     pop art, surrealism
2            Aaron Siskind             photography
3     Abbott Fuller Graves    impressionism, scene
4  Abbott Handerson Thayer  romanticism, symbolism
